# Creating and interacting with a DAXA Archive

This tutorial will explain the basic concepts behind the second type of important class in DAXA, the Archive class (with mission classes being the first type, see [the missions tutorial](missions.html)). 

DAXA Archives take a set of filtered missions, make sure that their data are downloaded, and enable easy access and organisation of all data files and processing functions. This includes storing the logs and errors of all processing steps (if run), and allowing for their easy retrieval. Much of DAXA's convenience and utility is in this functionality, as it will not only manage the myriad files produces during the processing, but keep track of which processes failed for which data, ensuring that any further processing is only run on data that has successfully passed through the dependancy processes.

Archives can also be loaded back into DAXA at a later date, so that the processing logs of data that has since been found to be problematic can be easily inspected, or indeed so that processing steps can be re-run with different settings; this also allows for archives to be updated, if more data become available.

## Import Statements

## What is a DAXA archive?

## Creating an archive from missions

## Loading an existing archive

## Archive properties

### ...

## 